In [53]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.feature_selection import RFE
import sklearn.preprocessing
import sklearn.decomposition
import sklearn.model_selection
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import sklearn.decomposition
import sklearn.model_selection
import sklearn.cluster
import yellowbrick.cluster
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [54]:
data = pd.read_csv("ClaMP_Train.csv")
data.fillna(0)

,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,e_csum,...,CheckSum,Subsystem,DllCharacteristics,SizeOfStackReserve,SizeOfStackCommit,SizeOfHeapReserve,SizeOfHeapCommit,LoaderFlags,NumberOfRvaAndSizes,class
0,23117,144,3,0,4,0,65535,0,184,0,...,65452,2,1024,1048576,4096,1048576,4096,0,16,0
1,23117,144,3,0,4,0,65535,0,184,0,...,0,2,0,1048576,4096,1048576,4096,0,16,1
2,23117,144,3,0,4,0,65535,0,184,0,...,119291,2,0,1048576,4096,1048576,4096,0,16,0
3,23117,144,3,0,4,0,65535,0,184,0,...,91168,3,320,262144,4096,1048576,4096,0,16,0
4,23117,144,3,0,4,0,65535,0,184,0,...,82656,2,32768,1048576,4096,1048576,4096,0,16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,23117,144,3,0,4,0,65535,0,184,0,...,4341763,2,33088,262144,8192,1048576,4096,0,16,0
2660,23117,144,3,0,4,0,65535,0,184,0,...,300418,3,320,262144,4096,1048576,4096,0,16,0
2661,23117,144,3,0,4,0,65535,0,184,0,...,51573,3,320,262144,4096,1048576,4096,0,16,0
2662,23117,144,3,0,4,0,65535,0,184,0,...,239267,2,0,1048576,4096,1048576,4096,0,16,0


In [3]:
data['class'].value_counts()

0    2001
1     663
Name: class, dtype: int64

In [55]:
drop_features = ["e_magic","e_crlc","e_res","e_res2"]
numerical_features = ["e_cblp","e_cp","e_cparhdr","e_minalloc","e_maxalloc","e_ss","e_sp","e_ip","e_cs","e_lfarlc","e_ovno","e_lfanew",
                      "NumberOfSections","CreationYear","PointerToSymbolTable","NumberOfSymbols","SizeOfOptionalHeader",
                      "SizeOfCode","SizeOfInitializedData","SizeOfUninitializedData","AddressOfEntryPoint","BaseOfCode","BaseOfData",
                      "ImageBase","SectionAlignment","FileAlignment",
                      "SizeOfImage","SizeOfHeaders","SizeOfStackReserve","SizeOfStackCommit","SizeOfHeapReserve",
                      "SizeOfHeapCommit","NumberOfRvaAndSizes"
                      ]
categorical_features = ["e_csum","e_oemid","e_oeminfo","Machine","Characteristics","Magic","CheckSum","DllCharacteristics",
                        "LoaderFlags","Subsystem","MajorOperatingSystemVersion",
                      "MinorOperatingSystemVersion","MajorImageVersion","MinorImageVersion","MajorSubsystemVersion","MinorSubsystemVersion",
                      "MajorLinkerVersion","MinorLinkerVersion"]

In [56]:
Target = data['class']
features_raw = data.drop(drop_features + ['class'], axis = 1)

In [58]:
scaler = MinMaxScaler()
num_features_scaled = pd.DataFrame(scaler.fit_transform(features_raw[numerical_features]), columns=features_raw[numerical_features].columns)


In [8]:
num_features_scaled.head()

,e_cblp,e_cp,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,e_ip,e_cs,e_lfarlc,...,ImageBase,SectionAlignment,FileAlignment,SizeOfImage,SizeOfHeaders,SizeOfStackReserve,SizeOfStackCommit,SizeOfHeapReserve,SizeOfHeapCommit,NumberOfRvaAndSizes
0,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0.125810,0.5,0.125,0.000202,0.001263,0.031250,0.001953,0.03125,0.0625,1.0
1,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0.001966,0.5,0.125,0.001412,0.001263,0.031250,0.001953,0.03125,0.0625,1.0
2,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0.222133,0.5,1.000,0.000655,0.005051,0.031250,0.001953,0.03125,0.0625,1.0
3,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0.125810,0.5,0.125,0.000655,0.001263,0.007812,0.001953,0.03125,0.0625,1.0
4,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0.001966,0.5,0.125,0.000756,0.001263,0.031250,0.001953,0.03125,0.0625,1.0


In [59]:
cat_features_raw = features_raw[categorical_features].copy()
for col in categorical_features:
  cat_features_raw[col] = cat_features_raw[col].astype("category")
one_hot_encoding = 'yes'
cat_features_final = pd.get_dummies(cat_features_raw, drop_first=True)
num_features_encoded_all = cat_features_final.shape[1]
print("{} total features after one-hot encoding.".format(num_features_encoded_all))

2215 total features after one-hot encoding.


In [60]:
# select categorical features from the dataset and set them as a category
cat_features_raw = features_raw[list(set(categorical_features)-set(["SizeOfCode","SizeOfInitializedData","AddressOfEntryPoint","BaseOfData","ImageBase","SizeOfImage","CheckSum"]))].copy()
for col in list(set(categorical_features)-set(["SizeOfCode","SizeOfInitializedData","AddressOfEntryPoint","BaseOfData","ImageBase","SizeOfImage","CheckSum"])):
                cat_features_raw[col] = cat_features_raw[col].astype("category")
# one hot encode the features and drop the first category
cat_features_final = pd.get_dummies(cat_features_raw, drop_first=True)

# Print the number of categorical features after one-hot encoding (with CheckSum removed)
num_features_encoded_final = cat_features_final.shape[1]
print("{} total features after one-hot encoding.".format(num_features_encoded_final))

163 total features after one-hot encoding.


In [61]:
# This combines and cleans the data and removes any nan values
features_final = pd.concat([num_features_scaled,cat_features_final],axis=1)

features_final = features_final.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
display(features_final.head(5))

,e_cblp,e_cp,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,e_ip,e_cs,e_lfarlc,...,Subsystem_1,Subsystem_2,Subsystem_3,Subsystem_9,Subsystem_16,e_csum_2,MinorSubsystemVersion_1,MinorSubsystemVersion_2,MinorSubsystemVersion_10,MinorSubsystemVersion_20
0,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0,1,0,0,0,0,0,0,0,0
1,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0,1,0,0,0,0,0,0,0,0
2,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0,1,0,0,0,0,0,0,0,0
3,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0,0,1,0,0,0,1,0,0,0
4,1.0,0.375,1.0,0.0,1.0,0.0,0.002808,0.0,0.0,1.0,...,0,1,0,0,0,0,0,0,0,0


In [62]:
# Split the 'features' and 'Target' data into training and testing sets using a train set that is 80% of the data and a random state of np.random.RandomState(seed)
X_train, X_test, y_train, y_test = train_test_split(features_final, Target, test_size=0.2, random_state=np.random.RandomState(100))

n_train_records = X_train.shape[0]
n_test_records = X_test.shape[0]

# Show the results of the split
print("Training set has {} samples.".format(n_train_records))
print("Testing set has {} samples.".format(n_test_records))

Training set has 2131 samples.
Testing set has 533 samples.


In [63]:
gradient = GradientBoostingClassifier(random_state=100, learning_rate=0.1, max_depth=3, n_estimators=100)#TODO
model = HistGradientBoostingClassifier()

In [64]:
gradient.fit(X_train, y_train)

GradientBoostingClassifier(random_state=100)

In [65]:
gradient_accuracy = accuracy_score(y_test,gradient.predict(X_test))
gradient_recall = recall_score(y_test,gradient.predict(X_test))
gradient_precision = precision_score(y_test,gradient.predict(X_test))
gradient_fscore = f1_score(y_test,gradient.predict(X_test))
gradient_roc_auc = roc_auc_score(y_test,gradient.predict_proba(X_test)[:,1])

In [20]:
print("roc auc =",  gradient_roc_auc)

roc auc = 0.9941294074881861


In [66]:
gfeature_importances = pd.DataFrame(gradient.feature_importances_, index =X_train.columns,  columns=['importance']).sort_values('importance', ascending=False)


In [67]:
display(gfeature_importances)

,importance
Characteristics_271,0.302594
ImageBase,0.119968
CreationYear,0.091980
Subsystem_2,0.062601
MajorImageVersion_6,0.052271
...,...
MinorLinkerVersion_14,0.000000
MinorLinkerVersion_15,0.000000
MinorLinkerVersion_20,0.000000
MinorLinkerVersion_22,0.000000


In [69]:
    names = X_train.columns.tolist()
    importance = gradient.feature_importances_
    df = pd.DataFrame({"Feature": names, "Importance": importance})
    df = df.sort_values(by="Importance", ascending=False)
    gb_importance = df.head(20)
    display(gb_importance)

,Feature,Importance
107,Characteristics_271,0.302594
23,ImageBase,0.119968
13,CreationYear,0.091980
187,Subsystem_2,0.062601
154,MajorImageVersion_6,0.052271
19,SizeOfUninitializedData,0.034621
12,NumberOfSections,0.029388
20,AddressOfEntryPoint,0.028798
121,Characteristics_8450,0.025155
37,MajorOperatingSystemVersion_6,0.023293


In [50]:
gb_importance = gb_importance.reset_index()
display(gb_importance)

,index,Feature,Importance
0,107,Characteristics_271,0.302594
1,23,ImageBase,0.119968
2,13,CreationYear,0.091980
3,187,Subsystem_2,0.062601
4,154,MajorImageVersion_6,0.052271
5,19,SizeOfUninitializedData,0.034621
6,12,NumberOfSections,0.029388
7,20,AddressOfEntryPoint,0.028798
8,121,Characteristics_8450,0.025155
9,37,MajorOperatingSystemVersion_6,0.023293


In [29]:
prediction_test = gradient.predict(X_test)

In [32]:
print(prediction_test)

[0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1
 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0
 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1
 0 1 1 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0
 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0 1
 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 1
 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 

In [34]:
probability_test = gradient.predict_proba(X_test)[:, 1]

In [36]:
print(gradient.predict_proba(X_test))

[[0.99498067 0.00501933]
 [0.99476735 0.00523265]
 [0.98153957 0.01846043]
 ...
 [0.14915216 0.85084784]
 [0.94792841 0.05207159]
 [0.99008443 0.00991557]]


In [37]:
print(gradient.predict_proba(X_test)[:, 1])

[0.00501933 0.00523265 0.01846043 0.61775102 0.97064596 0.70257268
 0.00763305 0.04120252 0.01517686 0.78025317 0.48398788 0.02010489
 0.01475406 0.37552765 0.01953113 0.95269446 0.01247783 0.00639802
 0.02945393 0.010783   0.0223348  0.14860405 0.00655734 0.25555545
 0.01659846 0.9795566  0.00286372 0.01493853 0.85431964 0.00245665
 0.00212953 0.89014734 0.02820644 0.01357774 0.01809453 0.96590927
 0.01563947 0.00763305 0.41352387 0.81679488 0.43824707 0.01020489
 0.02123084 0.01355193 0.31768405 0.01116436 0.01036099 0.00543734
 0.93979773 0.00783233 0.00245665 0.45814191 0.19293984 0.69495691
 0.061864   0.00594441 0.88520718 0.00859977 0.45795456 0.00447049
 0.09856585 0.02833809 0.01479791 0.01403594 0.0113679  0.85519567
 0.97607854 0.03099554 0.02205251 0.00490377 0.10851069 0.0200465
 0.05095668 0.98249522 0.00245665 0.11281176 0.30085126 0.74335846
 0.81679488 0.05740716 0.96063576 0.00807824 0.00763305 0.04589016
 0.03999881 0.01749066 0.89014734 0.16126804 0.16972142 0.94350

In [38]:
len(X_test)

533

In [40]:
np.arange(X_test)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [42]:
np.arange(len(X_test))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [45]:
index_value = np.arange(len(X_test))
score = gradient.predict_proba(X_test)[:, 1]
test_scores = pd.DataFrame({"index": index_value, "malware_score": score})

In [47]:
print(test_scores)

     index  malware_score
0        0       0.005019
1        1       0.005233
2        2       0.018460
3        3       0.617751
4        4       0.970646
..     ...            ...
528    528       0.013283
529    529       0.012716
530    530       0.850848
531    531       0.052072
532    532       0.009916

[533 rows x 2 columns]
